# CUHK-STAT3009: Homework 2 - More SVD Models **(due Nov 13)**


## **Q1: Basic Usage of SVD for Rating Prediction**

**Importing the SVD Class**

Download the `SVD` class from our GitHub repository: https://github.com/statmlben/CUHK-STAT3009/blob/main/src/TabRS.py.

**Dataset**

We will use a synthetic dataset to demonstrate the basic usage of SVD for rating prediction. The dataset consists of user ratings for various items, represented by the following DataFrame:
```python
import pandas as pd
data = {
    'user_id': [1, 1, 2, 2, 3, 3, 4, 4, 5, 5],
    'item_id': [1, 2, 1, 3, 1, 3, 2, 3, 2, 3],
    'rating': [5, 3, 4, 2, 1, 3, 4, 5, 2, 3]}
df = pd.DataFrame(data)
```

**Task**

Your task is to train an SVD model with $K = 2$ and $\lambda = 0.001$ using the provided dataset and predict the ratings for the following user-item pairs:

* `user_id` = 2, `item_id` = 2
* `user_id` = 5, `item_id` = 1

Implement the SVD model, train it on the dataset, and provide the predicted ratings for the specified user-item pairs.

> The correctness of the implementation will be evaluated based on the code structure and logic, not on the final evaluation results.

In [ ]:
!wget -q https://raw.githubusercontent.com/statmlben/CUHK-STAT3009/main/src/TabRS.py

In [ ]:
## Your solution here
import numpy as np
import pandas as pd
from TabRS import SVD

def rmse(y_true, y_pred):
    return np.sqrt(np.mean((y_true - y_pred)**2))

# 原始 synthetic 数据
data = {
    'user_id': [1, 1, 2, 2, 3, 3, 4, 4, 5, 5],
    'item_id': [1, 2, 1, 3, 1, 3, 2, 3, 2, 3],
    'rating': [5, 3, 4, 2, 1, 3, 4, 5, 2, 3]
}
df = pd.DataFrame(data)

# 🚩 关键一步：改成 0-based 索引（必须）
df["user_idx"] = df["user_id"] - 1   # 1~5 → 0~4
df["item_idx"] = df["item_id"] - 1   # 1~3 → 0~2

# 这里没有真正的 train/test，就按你习惯的变量名来
train = df.copy()
test = df.copy()

# 按你喜欢的格式写 X_train / X_test
X_train = train[["user_idx", "item_idx"]].values
y_train = train["rating"].values

X_test = test[["user_idx", "item_idx"]].values
y_test = test["rating"].values

# 仍然用 union 的写法算 n_users / n_items
n_users = len(set(X_train[:,0]).union(X_test[:,0]))  # = 5
n_items = len(set(X_train[:,1]).union(X_test[:,1]))  # = 3

model = SVD(
    n_users=n_users,
    n_items=n_items,
    K=2,
    lam=0.001
)

model.fit(X_train, y_train)

# 题目要求预测的两个 pair（先用原始 id 写）
pairs_raw = np.array([
    [2, 2],  # user_id=2, item_id=2
    [5, 1]   # user_id=5, item_id=1
])

# 同样要转成 0-based
pairs_idx = pairs_raw - 1   # [[1,1], [4,0]]

y_pred = model.predict(pairs_idx)

for (u, i), r in zip(pairs_raw, y_pred):
    print(f"user_id={u}, item_id={i} -> predicted rating = {r:.4f}")


Fitting Reg-SVD: K: 2, lam: 0.00100
RegSVD-ALS: 0; obj: 0.019; rmse:0.015, diff: 12.303
RegSVD-ALS: 1; obj: 0.018; rmse:0.014, diff: 0.000
RegSVD-ALS: 2; obj: 0.018; rmse:0.014, diff: 0.000
RegSVD-ALS: 3; obj: 0.017; rmse:0.013, diff: 0.000
RegSVD-ALS: 4; obj: 0.017; rmse:0.012, diff: 0.000
RegSVD-ALS: 5; obj: 0.017; rmse:0.012, diff: 0.000
RegSVD-ALS: 6; obj: 0.016; rmse:0.011, diff: 0.000
RegSVD-ALS: 7; obj: 0.016; rmse:0.011, diff: 0.000
RegSVD-ALS: 8; obj: 0.016; rmse:0.011, diff: 0.000
RegSVD-ALS: 9; obj: 0.016; rmse:0.010, diff: 0.000
user_id=2, item_id=2 -> predicted rating = 3.9555
user_id=5, item_id=1 -> predicted rating = 4.7349


## **Q2: Lasso-SVD Recommender Systems**

**Data**

In this task, you will implement a user-item average based recommender system using the Netflix dataset from the CUHK-STAT3009 GitHub repository.

```python
import numpy as np
import pandas as pd

# Load the Netflix dataset from the CUHK-STAT3009 GitHub repository
# Repository link: https://github.com/statmlben/CUHK-STAT3009/tree/main/dataset/netflix

train = pd.read_csv('https://raw.githubusercontent.com/statmlben/CUHK-STAT3009/main/dataset/netflix/train.csv')
test = pd.read_csv('https://raw.githubusercontent.com/statmlben/CUHK-STAT3009/main/dataset/netflix/test.csv')

# Convert DataFrame to NumPy arrays
```

**Lasso Regression**

Given a dataset of feature-vectors $\mathbf{x}_i$ and corresponding ground truth scores $y_i$, Lasso regression seeks a sparse solution by minimizing the following objective function:

$$\text{argmin}_{\mathbf{\beta}} \ \frac{1}{n} \sum_{i=1}^n ( y_i - \mathbf{\beta}^T \mathbf{x}_i )^2 + \lambda \| \mathbf{\beta} \|_1, \quad \text{where } \| \mathbf{\beta} \|_1 = \sum_{j=1}^p |\beta_j|$$

This can be efficiently solved using `sklearn.linear_model.Lasso` ([documentation](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html#sklearn.linear_model.Lasso)).

### **Task: Lasso Matrix Factorization (Lasso_SVD)**

**Objective**

Implement a Lasso_SVD recommender system by solving the following optimization problem:

$$\boxed{(\widehat{\mathbf P}, \widehat{\mathbf Q}) = \text{argmin}_{\mathbf{P}, \mathbf{Q} } \frac{1}{|\Omega|} \sum_{(u,i) \in \Omega} ( r_{ui} - \mathbf{p}^\intercal_u \mathbf{q}_i  )^2 + \lambda \big(  \sum_{u=1}^n \|\mathbf{p}_u\|_1 + \sum_{i=1}^m \|\mathbf{q}_i\|_1 \big)}$$

**Implementation**

Create a class `Lasso_SVD` with two methods:

1. `Lasso_SVD.fit`: Fit the parameters $\mathbf{P}$ and $\mathbf{Q}$ by solving the optimization problem above using Lasso regression.
2. `Lasso_SVD.predict`: Predict ratings using the fitted parameters: $\widehat{r}_{ui} = \widehat{\mathbf{p}}^T_u \widehat{\mathbf{q}}_i$

**Hint**: Use Alternative Least Square (ALS) logic, where each subproblem is a Lasso regression that can be solved using `sklearn.linear_model.Lasso` (previously, we use `sklearn.linear_model.Ridge`).

**Evaluation**

Print the Root Mean Squared Error (RMSE) for the testing data using the following hyperparameters:

* $(\lambda = 0.1, K = 3)$
* $(\lambda = 0.3, K = 5)$

> Implement the `Lasso_SVD` class with the required methods. The correctness of the implementation will be evaluated based on the code structure and logic, not on the final evaluation results.

In [ ]:
## Your solution here
import numpy as np
import pandas as pd

train = pd.read_csv('https://raw.githubusercontent.com/statmlben/CUHK-STAT3009/main/dataset/netflix/train.csv')
test = pd.read_csv('https://raw.githubusercontent.com/statmlben/CUHK-STAT3009/main/dataset/netflix/test.csv')
train.head()
X_train = train[["user_id", "movie_id"]].values
y_train = train["rating"].values
X_test = test[["user_id", "movie_id"]].values
y_test = test["rating"].values
n_users = len(set(X_train[:,0]).union(X_test[:,0]))
n_items = len(set(X_train[:,1]).union(X_test[:,1]))


In [ ]:
from sklearn.linear_model import Lasso,Ridge
from sklearn.base import BaseEstimator
from sklearn.metrics import mean_squared_error
class Lasso_SVD(BaseEstimator):
  def __init__(self, n_user, n_item, K=5, lam=0.01, epsilon=1, iterNum=10, tol=1e-4):
    self.n_user = n_user
    self.n_item = n_item

    self.K = K
    self.lam = lam
    self.P = np.random.randn(n_user, K)
    self.Q = np.random.randn(n_item, K)
    self.iterNum = iterNum
    self.tol = tol

  def fit(self, X, y):
    n_obs = len(X)
    diff = 1.0
    obj_tmp = self.obj(X, y)

    for h in range(self.iterNum):
      if diff < self.tol:
        break
      for i in range(self.n_item):
        index_tmp = np.where(X[:,1] == i)[0]
        if len(index_tmp) == 0:
          self.Q[i] = np.zeros(self.K)
        else:
          y_tmp = y[index_tmp]
          P_tmp = self.P[X[index_tmp,0]]
          clf_tmp = Lasso(alpha=self.lam*n_obs, fit_intercept=False)
          clf_tmp.fit(X=P_tmp, y=y_tmp)
          self.Q[i] = clf_tmp.coef_

      for u in range(self.n_user):
        index_tmp = np.where(X[:,0] == u)[0]
        if len(index_tmp) == 0:
          self.P[u] = np.zeros(self.K)
        else:
          y_tmp = y[index_tmp]
          Q_tmp = self.Q[X[index_tmp,1]]
          clf_tmp = Lasso(alpha=self.lam*n_obs,fit_intercept=False)
          clf_tmp.fit(X=Q_tmp, y=y_tmp)
          self.P[u] = clf_tmp.coef_

      obj_old = obj_tmp
      obj_tmp = self.obj(X, y)
      diff = obj_old - obj_tmp

      print(f"Lasso_SVD-BCD: step: {h}; obj: {obj_tmp}, diff: {diff}")
    return self

  def predict(self, X):
    y_pred = []
    for x_tmp in X:
      user_id_tmp = x_tmp[0]
      item_id_tmp = x_tmp[1]

      y_pred_tmp = self.P[user_id_tmp] @ self.Q[item_id_tmp]
      y_pred.append(y_pred_tmp)
    return np.array(y_pred)

  def obj(self, X, y):
    y_pred = self.predict(X)
    mse = mean_squared_error(y,y_pred)
    pen = np.sum(self.P**2) + np.sum(self.Q**2)
    return mse + self.lam * pen


In [ ]:
model1= Lasso_SVD(n_users, n_items, K=3, lam=0.1)
model2= Lasso_SVD(n_users, n_items, K=5, lam=0.3)
model1.fit(X_train, y_train)
y_pred1 = model1.predict(X_test)
model2.fit(X_train, y_train)
y_pred2 = model2.predict(X_test)
print(f'RMSE for (λ=0.1,K=3): {rmse(y_pred1, y_test)}')
print(f'RMSE for (λ=0.3,K=5): {rmse(y_pred2, y_test)}')

Lasso_SVD-BCD: step: 0; obj: 14.296202185258302, diff: 1674.976982855409
Lasso_SVD-BCD: step: 1; obj: 14.296202185258302, diff: 0.0
Lasso_SVD-BCD: step: 0; obj: 14.296202185258302, diff: 8513.790390942793
Lasso_SVD-BCD: step: 1; obj: 14.296202185258302, diff: 0.0
RMSE for (λ=0.1,K=3): 3.7870676665868332
RMSE for (λ=0.3,K=5): 3.7870676665868332


## **Q3: Kaggle Submission by Neural Networks**

**Task**

- Create an arbitrary Neural Network with Dense layers and Make a Submission to the Kaggle Competition: [House Prices - Advanced Regression Techniques](https://www.kaggle.com/competitions/house-prices-advanced-regression-techniques/overview)

- Paste the submission results screenshot into this Jupyter Notebook.

In [ ]:
## Your submission result here

## **Q4 (Bonus): Parallel Alternating Least Squares (ALS) for Matrix Factorization**


**Background**

Recall the item and user updates for `SVD` based on ALS:

$$\mathbf{q}^{(l+1)}_i = \left(\sum_{u \in \mathcal{U}_i} \mathbf{p}^{(l)}_u (\mathbf{p}^{(l)}_u)^T + \lambda |\Omega| \mathbf{I}\right)^{-1} \sum_{u \in \mathcal{U}_i} r_{ui} \mathbf{p}^{(l)}_u$$

$$\mathbf{p}^{(l+1)}_u = \left(\sum_{i \in \mathcal{I}_u} \mathbf{q}^{(l+1)}_i (\mathbf{q}^{(l+1)}_i)^\intercal + \lambda |\Omega| \mathbf{I}\right)^{-1} \sum_{i \in \mathcal{I}_u} r_{ui} \mathbf{q}^{(l+1)}_i$$

The key observation is that the updates for user-$u$ and item-$i$ are independent of other users and items, respectively. Therefore, they can be performed in parallel.

Suppose you have 100 users to update, the basic ALS updates user 1, user 2, ..., user 100 sequentially in a loop. Now, suppose you have 100 CPUs, the parallel ALS can update 100 users simultaneously by distributing each user to a different CPU, significantly reducing the computation time.

**Tasks**

1. **Parallelize the `SVD.fit` method**: Revise the `SVD.fit` method (available in [repo](https://github.com/statmlben/CUHK-STAT3009/blob/main/src/TabRS.py)) to allow parallel updating of $\mathbf{p}_u$ and $\mathbf{q}_i$ using Python libraries such as [multiprocessing](https://docs.python.org/3/library/multiprocessing.html) or [pymp](https://github.com/classner/pymp).
2. **Compare computation times**: Compare the computation time for `SVD.fit` with and without parallel computing using the `%%time` magic command (see [ref](https://stackoverflow.com/questions/32565829/simple-way-to-measure-cell-execution-time-in-ipython-notebook)).

In [ ]:
# check number of CPUs in your PC/Node
!lscpu

Architecture:        x86_64
CPU op-mode(s):      32-bit, 64-bit
Byte Order:          Little Endian
CPU(s):              2
On-line CPU(s) list: 0,1
Thread(s) per core:  2
Core(s) per socket:  1
Socket(s):           1
NUMA node(s):        1
Vendor ID:           GenuineIntel
CPU family:          6
Model:               79
Model name:          Intel(R) Xeon(R) CPU @ 2.20GHz
Stepping:            0
CPU MHz:             2199.998
BogoMIPS:            4399.99
Hypervisor vendor:   KVM
Virtualization type: full
L1d cache:           32K
L1i cache:           32K
L2 cache:            256K
L3 cache:            56320K
NUMA node0 CPU(s):   0,1
Flags:               fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_sin

In [ ]:
## You solution here